In [ ]:
# -*- coding: utf-8 -*-
"""


@author: Paolo Musone
"""
# import pandas 
import pandas as pd
# must import bt to use bt
import bt

import pandas_datareader.data as web
import seaborn as sn
import matplotlib.pyplot as plt


def tree_mom(tickers,crypto, asset,m,n, beginning='2018-01-01', name='master_str'):
   
    # download data
    data = bt.get(tickers, start=beginning, end='2021-06-01')
    

    # create strategy  for mom_s
    s = bt.Strategy('mom_s', [bt.algos.RunMonthly(),
                                  bt.algos.SelectAll(),
                                  bt.algos.SelectMomentum(m),
                                  bt.algos.WeighEqually(),
                                  bt.algos.Rebalance()],crypto)
    
    # create strategy  for z_mom_s
    z_s = bt.Strategy('z_mom_s', [bt.algos.RunMonthly(),
                                  bt.algos.SelectAll(),
                                  bt.algos.SelectMomentum(n),
                                  bt.algos.WeighEqually(),
                                  bt.algos.Rebalance()],asset)
    
    master = bt.Strategy(name, [bt.algos.RunMonthly(),
                                        bt.algos.SelectAll(),
                                        bt.algos.WeighEqually(),
                                        bt.algos.Rebalance()],
                            [s,z_s])

    # now we create the backtest
    return bt.Backtest(master, data)


# Create a result_df to store results
result_df = pd.DataFrame(columns = ['mom_c','mom_s','CAGR (%)','Daily Sharpe','Total Return (%)'])
#result_df = pd.DataFrame(columns = ['mom_s','z_mom_s','CAGR (%)','daily_sharpe'])


# Our simple investment universe 
asset_list = ['lnt', 'aee', 'all','amt','apa','t','ato','abb']
crypto_list = ['btc-usd','eth-usd','usdt-usd','bnb-usd','DX-Y.NYB']
equity_list = ['lnt', 'aee', 'all','amt','apa','t','ato','abb','btc-usd','eth-usd','usdt-usd','bnb-usd','DX-Y.NYB'] 




for mom_m in range(1, len(crypto_list)+1):
    
    for mom_n in range (1, len(asset_list)+1):
        # create a string that is the name of our master strategy
        t_mom_name='t_mom'+ str(mom_m) + ','+ str(mom_n)
    
        # generate result feeding the above_sma function variables that change in the loop
        result = bt.run(tree_mom(tickers=equity_list, crypto=crypto_list,asset=asset_list, name= t_mom_name,m=mom_m,n=mom_n))

        # There is a more elegant way to do this, but this works
        result_df = result_df.append({'mom_c':mom_m,'mom_s':mom_n,'CAGR (%)':result.stats.at['cagr',t_mom_name]*100, 
                                  'Daily Sharpe': result.stats.at['daily_sharpe',t_mom_name],
                                  'Total Return (%)': result.stats.at['total_return',t_mom_name]*100},ignore_index=True)

max = result_df.iloc[result_df['CAGR (%)'].idxmax()]
print('max:', max)

min = result_df.iloc[result_df['CAGR (%)'].idxmin()]
print('min:', min)

result_df.sort_values(['CAGR (%)'])



# Set the index to momentum. The other arguments are there because StackOverflow suggested them and they work
#result_df.set_index('mom_s',inplace=True, drop=True)
"""
# Plot the better result
best_result = bt.run(tree_mom(tickers=equity_list, crypto=crypto_list,asset=asset_list,m=mom_m,n=mom_n))


#sn.heatmap(result_df, annot=True)
#plt.show()

result.plot()
# Show some performance metrics
result.display()
result.plot_security_weights()"""